In [27]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import datetime as dt
import selenium
import re
# test out import urllib
browser = Browser("chrome", executable_path="chromedriver", headless=True)
#URL's    
# mars news url = "https://mars.nasa.gov/news/"
# mars images url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
# Hemishpere Url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
#Weather url = "https://twitter.com/marswxreport?lang=en"

In [37]:
# News

news_url = "https://mars.nasa.gov/news/"
browser.visit(news_url)

browser.is_element_present_by_css("ul.item_list li.slide", wait_time=0.5)
html = browser.html
PrettyNews = BeautifulSoup(html, "html.parser")

slide_elem = PrettyNews.select_one("ul.item_list li.slide")
news_title = slide_elem.find("div", class_="content_title").get_text()
news_p = slide_elem.find("div", class_="article_teaser_body").get_text()

print(news_p)
print(news_title)

Observing the Martian moons Phobos and Deimos crossing in front of the Sun helps scientists better understand their orbits.
Curiosity Captured Two Solar Eclipses on Mars


In [38]:
image_url="https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(image_url)

full_image_elem = browser.find_by_id("full_image")
full_image_elem.click()

browser.is_element_present_by_text("more info", wait_time=0.5)
more_info_elem = browser.find_link_by_partial_text("more info")
more_info_elem.click()

html = browser.html
img_soup = BeautifulSoup(html, "html.parser")

img = img_soup.select_one("figure.lede a img")

img_url_rel = img.get("src")

img_url = f"https://www.jpl.nasa.gov{img_url_rel}"

img_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA09113_hires.jpg'

In [41]:
hemisphere_url = ("https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars")
browser.visit(hemisphere_url)

hemisphere_image_urls = []

links = browser.find_by_css("a.product-item h3")

for i in range(len(links)):
    hemisphere = {}
    
    browser.find_by_css("a.product-item h3")[i].click()
    
    sample_elem = browser.find_link_by_text('Sample').first
    hemisphere['img_url'] = sample_elem['href']
    
    hemisphere['title'] = browser.find_by_css("h2.title").text
    
    hemisphere_image_urls.append(hemisphere)
    
    browser.back()
print(hemisphere_image_urls)

[{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg', 'title': 'Cerberus Hemisphere Enhanced'}, {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg', 'title': 'Schiaparelli Hemisphere Enhanced'}, {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg', 'title': 'Syrtis Major Hemisphere Enhanced'}, {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg', 'title': 'Valles Marineris Hemisphere Enhanced'}]


In [43]:
twitter_url = "https://twitter.com/marswxreport?lang=en"
browser.visit(twitter_url)

html = browser.html
weather_soup = BeautifulSoup(html, "html.parser")

tweet_attrs = {"class": "tweet", "data-name": "Mars Weather"}
mars_weather_tweet = weather_soup.find("div", attrs=tweet_attrs)

mars_weather = mars_weather_tweet.find("p", "tweet-text").get_text()

mars_weather

'InSight sol 125 (2019-04-03) low -97.2ºC (-143.0ºF) high -16.8ºC (1.7ºF)\nwinds from the SW at 4.0 m/s (8.9 mph) gusting to 11.7 m/s (26.2 mph)\npressure at 7.30 hPapic.twitter.com/ht1lEraC6M'

In [44]:
df = pd.read_html("http://space-facts.com/mars/")[0]

df.columns = ["description", "value"]
df.set_index("description", inplace=True)
    
df

,value
description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [47]:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>value</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>

In [48]:
browser.quit()